In [1]:
!pip install requests pandas pymongo Flask pyngrok

You should consider upgrading via the 'D:\python3.9\python.exe -m pip install --upgrade pip' command.


In [2]:
# API connection
import requests

def fetch_data(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    return response.json()

# URL for the online database
api_url = 'https://data.cityofnewyork.us/resource/ic3t-wcy2.json'
data = fetch_data(api_url)

In [3]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(data)

/Users/a333/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
# Data transformation
df = df.iloc[:, :13] #Keep only the most important columns
print(df)

         job__ doc__    borough house__        street_name  block    lot  \
0    301776077    01   BROOKLYN     208  NORTH    8 STREET  02321  00008   
1    301776291    01   BROOKLYN     933          51 STREET  05645  00066   
2    302163557    01   BROOKLYN      96     ROCKWELL PLACE  02106  07501   
3    440673852    01     QUEENS  246-08           135 ROAD  13228  00009   
4    421789559    01     QUEENS   98-87        QUEENS BLVD  02105  00001   
..         ...   ...        ...     ...                ...    ...    ...   
995  121324147    04  MANHATTAN     133   GREENWICH STREET  00052  00008   
996  120195154    01  MANHATTAN      12     WEST 19 STREET  00820  00053   
997  301942138    01   BROOKLYN     111     STEUBEN STREET  01894  00058   
998  121819577    01  MANHATTAN     734           BROADWAY  00545  00021   
999  421743297    01     QUEENS   35-06       UNION STREET  04961  00019   

       bin__ job_type job_status                job_status_descrp  \
0    3061967      

In [6]:
from pymongo import MongoClient

# Connect to MongoDB (adjust the connection string as necessary)
client = MongoClient('mongodb://localhost:27017/')
db = client['5400']  
collection = db['Job2'] 

In [7]:
# Delete existing documents to avoid repetition
collection.delete_many({})

# Convert DataFrame to dictionary format suitable for MongoDB
data_dict = df.to_dict('records')
# Insert data into the collection
collection.insert_many(data_dict)

InsertManyResult([ObjectId('6620262a914995a751b5009c'), ObjectId('6620262a914995a751b5009d'), ObjectId('6620262a914995a751b5009e'), ObjectId('6620262a914995a751b5009f'), ObjectId('6620262a914995a751b500a0'), ObjectId('6620262a914995a751b500a1'), ObjectId('6620262a914995a751b500a2'), ObjectId('6620262a914995a751b500a3'), ObjectId('6620262a914995a751b500a4'), ObjectId('6620262a914995a751b500a5'), ObjectId('6620262a914995a751b500a6'), ObjectId('6620262a914995a751b500a7'), ObjectId('6620262a914995a751b500a8'), ObjectId('6620262a914995a751b500a9'), ObjectId('6620262a914995a751b500aa'), ObjectId('6620262a914995a751b500ab'), ObjectId('6620262a914995a751b500ac'), ObjectId('6620262a914995a751b500ad'), ObjectId('6620262a914995a751b500ae'), ObjectId('6620262a914995a751b500af'), ObjectId('6620262a914995a751b500b0'), ObjectId('6620262a914995a751b500b1'), ObjectId('6620262a914995a751b500b2'), ObjectId('6620262a914995a751b500b3'), ObjectId('6620262a914995a751b500b4'), ObjectId('6620262a914995a751b500

In [10]:
# norok configuration
!ngrok authtoken 2fEnwsm3CxXjygvnMnGtOe6rmjS_4ZFFGsSpordiAqL4UgiB2

Authtoken saved to configuration file: /Users/a333/Library/Application Support/ngrok/ngrok.yml


In [11]:
# Return the ngrok URL for external access
def run_app():
    ngrok.set_auth_token('2fEnwsm3CxXjygvnMnGtOe6rmjS_4ZFFGsSpordiAqL4UgiB2')
    public_url = ngrok.connect(5001)
    print(f"ngrok tunnel 'public_url' -> 'http://127.0.0.1:5001/'")
    app.run(port=5001, use_reloader=False)
    return public_url

In [12]:
from flask import Flask, request, render_template
from pymongo import MongoClient
from pyngrok import ngrok
import threading

app = Flask(__name__, template_folder='.')

# MongoDB setup
client = MongoClient('mongodb://localhost:27017/')
db = client['5400']
collection = db.Job2

# Display the 'index.html' template on the homepage
@app.route('/', methods=['GET'])
def home():
    return render_template('index.html')

# Design the search interface
@app.route('/search', methods=['POST'])
def search():
    job_id = request.form.get('job_id')
    borough = request.form.get('borough')

    query = {}
    if job_id:
        query['job__'] = job_id
    if borough:
        query['borough'] = borough

    results = collection.find(query)
    results = list(results)  # Convert cursor to list for easier handling in the template
    return render_template('results.html', results=results)

# Run the Flask app
public_url = None #Store the norok URL
def run_app():
    global public_url
    ngrok.set_auth_token('2fEnwsm3CxXjygvnMnGtOe6rmjS_4ZFFGsSpordiAqL4UgiB2')
    public_url = ngrok.connect(5001)
    print(f"ngrok tunnel public URL is {public_url}")
    app.run(port=5001, use_reloader=False)

# Terminate the Flask app
def stop_flask():
    if public_url:
        ngrok.disconnect(public_url)
        print("ngrok tunnel closed")

In [13]:
# Initialize and start a separate thread
flask_thread = threading.Thread(target=run_app)
flask_thread.start()

In [14]:
stop_flask()

ngrok tunnel public URL is NgrokTunnel: "https://ef4b-209-2-225-5.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2024 15:42:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2024 15:42:54] "POST /search HTTP/1.1" 200 -
